In [85]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [29]:
# preview files

raw_prior_orders = pd.read_csv('../data/raw/order_products_prior.csv')
raw_train_orders = pd.read_csv('../data/raw/order_products_train.csv')
raw_products = pd.read_csv('../data/raw/products.csv')
raw_aisles = pd.read_csv('../data/raw/aisles.csv')
raw_departments = pd.read_csv('../data/raw/departments.csv')
order_prior_train_or_test = pd.read_csv('../data/raw/orders.csv')

In [30]:
raw_prior_orders.info()
raw_prior_orders.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 4 columns):
 #   Column             Dtype
---  ------             -----
 0   order_id           int64
 1   product_id         int64
 2   add_to_cart_order  int64
 3   reordered          int64
dtypes: int64(4)
memory usage: 989.8 MB


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [31]:
raw_train_orders.info()
raw_train_orders.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1384617 entries, 0 to 1384616
Data columns (total 4 columns):
 #   Column             Non-Null Count    Dtype
---  ------             --------------    -----
 0   order_id           1384617 non-null  int64
 1   product_id         1384617 non-null  int64
 2   add_to_cart_order  1384617 non-null  int64
 3   reordered          1384617 non-null  int64
dtypes: int64(4)
memory usage: 42.3 MB


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [32]:
raw_products.info()
raw_products.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49688 entries, 0 to 49687
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_id     49688 non-null  int64 
 1   product_name   49688 non-null  object
 2   aisle_id       49688 non-null  int64 
 3   department_id  49688 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.5+ MB


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [33]:
raw_aisles.info()
raw_aisles.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   aisle_id  134 non-null    int64 
 1   aisle     134 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.2+ KB


,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [34]:
raw_departments.info()
raw_departments.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   department_id  21 non-null     int64 
 1   department     21 non-null     object
dtypes: int64(1), object(1)
memory usage: 464.0+ bytes


,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [35]:
order_prior_train_or_test.info()
order_prior_train_or_test.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421083 entries, 0 to 3421082
Data columns (total 7 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   eval_set                object 
 3   order_number            int64  
 4   order_dow               int64  
 5   order_hour_of_day       int64  
 6   days_since_prior_order  float64
dtypes: float64(1), int64(5), object(1)
memory usage: 182.7+ MB


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [37]:
# Learn more about what I'm assuming are nested relationships among
# departments > aisles > products

raw_departments['department'].unique()

array(['frozen', 'other', 'bakery', 'produce', 'alcohol', 'international',
       'beverages', 'pets', 'dry goods pasta', 'bulk', 'personal care',
       'meat seafood', 'pantry', 'breakfast', 'canned goods',
       'dairy eggs', 'household', 'babies', 'snacks', 'deli', 'missing'],
      dtype=object)

In [38]:
raw_aisles['aisle'].unique()

array(['prepared soups salads', 'specialty cheeses',
       'energy granola bars', 'instant foods',
       'marinades meat preparation', 'other', 'packaged meat',
       'bakery desserts', 'pasta sauce', 'kitchen supplies',
       'cold flu allergy', 'fresh pasta', 'prepared meals',
       'tofu meat alternatives', 'packaged seafood', 'fresh herbs',
       'baking ingredients', 'bulk dried fruits vegetables',
       'oils vinegars', 'oral hygiene', 'packaged cheese', 'hair care',
       'popcorn jerky', 'fresh fruits', 'soap', 'coffee', 'beers coolers',
       'red wines', 'honeys syrups nectars', 'latino foods',
       'refrigerated', 'packaged produce', 'kosher foods',
       'frozen meat seafood', 'poultry counter', 'butter',
       'ice cream ice', 'frozen meals', 'seafood counter',
       'dog food care', 'cat food care', 'frozen vegan vegetarian',
       'buns rolls', 'eye ear care', 'candy chocolate', 'mint gum',
       'vitamins supplements', 'breakfast bars pastries',
       '

In [43]:
raw_products['aisle_id'].nunique()

134

In [44]:
raw_products['department_id'].nunique()

21

In [46]:
# Merge products, aisles, departments DataFrames

products_w_aisles = raw_products.merge(raw_aisles, on='aisle_id')

products_w_aisles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49688 entries, 0 to 49687
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_id     49688 non-null  int64 
 1   product_name   49688 non-null  object
 2   aisle_id       49688 non-null  int64 
 3   department_id  49688 non-null  int64 
 4   aisle          49688 non-null  object
dtypes: int64(3), object(2)
memory usage: 2.3+ MB


In [49]:
prod_aisle_dept = products_w_aisles.merge(raw_departments, on='department_id')

prod_aisle_dept.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49688 entries, 0 to 49687
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_id     49688 non-null  int64 
 1   product_name   49688 non-null  object
 2   aisle_id       49688 non-null  int64 
 3   department_id  49688 non-null  int64 
 4   aisle          49688 non-null  object
 5   department     49688 non-null  object
dtypes: int64(3), object(3)
memory usage: 2.7+ MB


In [64]:
# Change column names and order to make it look how I want

pad_rearranged = prod_aisle_dept.iloc[:, [0, 1, 4, 5, 2, 3]]

pad_rearranged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49688 entries, 0 to 49687
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_id     49688 non-null  int64 
 1   product_name   49688 non-null  object
 2   aisle          49688 non-null  object
 3   department     49688 non-null  object
 4   aisle_id       49688 non-null  int64 
 5   department_id  49688 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 3.7+ MB


In [65]:
pad = pad_rearranged

pad = pad.rename(columns={'aisle': 'aisle_name', 'department': 'dept_name'})

pad.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49688 entries, 0 to 49687
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_id     49688 non-null  int64 
 1   product_name   49688 non-null  object
 2   aisle_name     49688 non-null  object
 3   dept_name      49688 non-null  object
 4   aisle_id       49688 non-null  int64 
 5   department_id  49688 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 3.7+ MB


In [66]:
# Merge prior orders and training orders with pad (products, aisles, departments) df

prior_orders = raw_prior_orders.merge(pad, on='product_id')

prior_orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32434489 entries, 0 to 32434488
Data columns (total 9 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   order_id           int64 
 1   product_id         int64 
 2   add_to_cart_order  int64 
 3   reordered          int64 
 4   product_name       object
 5   aisle_name         object
 6   dept_name          object
 7   aisle_id           int64 
 8   department_id      int64 
dtypes: int64(6), object(3)
memory usage: 2.4+ GB


In [103]:
training_orders = raw_train_orders.merge(pad, on='product_id')

training_orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1384617 entries, 0 to 1384616
Data columns (total 9 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   order_id           1384617 non-null  int64 
 1   product_id         1384617 non-null  int64 
 2   add_to_cart_order  1384617 non-null  int64 
 3   reordered          1384617 non-null  int64 
 4   product_name       1384617 non-null  object
 5   aisle_name         1384617 non-null  object
 6   dept_name          1384617 non-null  object
 7   aisle_id           1384617 non-null  int64 
 8   department_id      1384617 non-null  int64 
dtypes: int64(6), object(3)
memory usage: 105.6+ MB


In [104]:
# Explore prior and training dfs to see if it could make sense to combine them
# I'm not sure whether it will be most useful to work with the original chunks or the entire available dataset, 
# but no harm in having options. 

unique_prior_ids = prior_orders['order_id'].unique()
unique_training_ids = training_orders['order_id'].unique()

unique_prior_ids.sort()
unique_training_ids.sort()

print(unique_prior_ids[0:50], unique_training_ids[0:50])

[ 2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 18 19 20 21 22 23 24 25 26
 27 28 29 30 31 32 33 35 37 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55] [   1   36   38   96   98  112  170  218  226  349  393  456  473  631
  719  762  774  844  878  904  915  988 1001 1032 1042 1077 1086 1119
 1120 1139 1143 1145 1275 1280 1318 1325 1335 1342 1350 1468 1571 1572
 1579 1591 1597 1620 1674 1682 1703 1721]


In [105]:
prior_order_list = unique_prior_ids.tolist()
training_order_list = unique_training_ids.tolist()

In [106]:
all_orders_list = prior_order_list + training_order_list
all_orders_list.sort()
print(all_order_list[0:50])

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52]


In [107]:
len(all_orders_list)

3346083

In [108]:
len(set(all_orders_list))

3346083

In [114]:
# There aren't duplicate order ids, so concatenating can be simple

all_orders = pd.concat([prior_orders, training_orders])

all_orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33819106 entries, 0 to 1384616
Data columns (total 9 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   order_id           int64 
 1   product_id         int64 
 2   add_to_cart_order  int64 
 3   reordered          int64 
 4   product_name       object
 5   aisle_name         object
 6   dept_name          object
 7   aisle_id           int64 
 8   department_id      int64 
dtypes: int64(6), object(3)
memory usage: 2.5+ GB


In [120]:
# It's ambiguous to me what add_to_cart_order means, so I went to look it up
# Found an answer at https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b
# Apparently it's the order in which an item was placed in a cart relative to other items for that order
# Not the order number of some prior order when the item was first purchased
# Which I could confirm by looking at details for any given order with multiple items

all_orders_sorted = all_orders.sort_values('order_id')

all_orders_sorted.head(10)

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_name,dept_name,aisle_id,department_id
26424,1,22035,8,1,Organic Whole String Cheese,packaged cheese,dairy eggs,21,16
19131,1,47209,7,0,Organic Hass Avocado,fresh fruits,produce,24,4
3651,1,13176,6,0,Bag of Organic Bananas,fresh fruits,produce,24,4
3627,1,43633,5,1,Lightly Smoked Sardines in Olive Oil,canned meat seafood,canned goods,95,15
1214,1,49683,4,0,Cucumber Kirby,fresh vegetables,produce,83,4
152,1,10246,3,0,Organic Celery Hearts,fresh vegetables,produce,83,4
8,1,11109,2,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,other creams cheeses,dairy eggs,108,16
0,1,49302,1,1,Bulgarian Yogurt,yogurt,dairy eggs,120,16
0,2,33120,1,1,Organic Egg Whites,eggs,dairy eggs,86,16
86849,2,9327,3,0,Garlic Powder,spices seasonings,pantry,104,13


In [121]:
# Clarifying the add_to_cart_order column name

all_orders = all_orders.rename(columns={'add_to_cart_order': 'add_to_cart_sequence'})
prior_orders = prior_orders.rename(columns={'add_to_cart_order': 'add_to_cart_sequence'})
training_orders = training_orders.rename(columns={'add_to_cart_order': 'add_to_cart_sequence'})

In [126]:
all_orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33819106 entries, 0 to 1384616
Data columns (total 9 columns):
 #   Column                Dtype 
---  ------                ----- 
 0   order_id              int64 
 1   product_id            int64 
 2   add_to_cart_sequence  int64 
 3   reordered             int64 
 4   product_name          object
 5   aisle_name            object
 6   dept_name             object
 7   aisle_id              int64 
 8   department_id         int64 
dtypes: int64(6), object(3)
memory usage: 2.5+ GB


In [127]:
# Dealing with missing values

all_orders.isnull().values.any()

False

In [130]:
# Begin to explore summary statistics and just better understand the data

all_orders['product_name'].describe()

count     33819106
unique       49685
top         Banana
freq        491291
Name: product_name, dtype: object

In [132]:
all_orders['aisle_name'].describe()

count         33819106
unique             134
top       fresh fruits
freq           3792661
Name: aisle_name, dtype: object

In [133]:
all_orders['dept_name'].describe()

count     33819106
unique          21
top        produce
freq       9888378
Name: dept_name, dtype: object

In [134]:
all_orders['reordered'].describe()

count    3.381911e+07
mean     5.900617e-01
std      4.918220e-01
min      0.000000e+00
25%      0.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      1.000000e+00
Name: reordered, dtype: float64

In [ ]:
# .describe isn't so interesting for the boolean-as-int column 'reordered,' though 
# the mean being 0.59 means that on average 60% of items in any order are reorders from sometime in the past

In [143]:
all_orders['add_to_cart_sequence'].describe()

count    3.381911e+07
mean     8.367738e+00
std      7.139540e+00
min      1.000000e+00
25%      3.000000e+00
50%      6.000000e+00
75%      1.100000e+01
max      1.450000e+02
Name: add_to_cart_sequence, dtype: float64

In [139]:
# Likewise, the description of 'add_to_cart_sequence' isn't super useful because smaller numbers are over-represented.
# Each order of n number of items will have add_to_cart_sequence values 1, 2,...n 
# It would be more interesting to find the smallest, largest, and average order sizes

all_orders.groupby('order_id')['add_to_cart_sequence'].max().min()

1

In [140]:
all_orders.groupby('order_id')['add_to_cart_sequence'].max().max()

145

In [141]:
all_orders.groupby('order_id')['add_to_cart_sequence'].max().mean()

10.10707325550502

In [175]:
# Peek at aisles per department

sorted_pad = pad.sort_values(['dept_name', 'aisle_name', 'product_name'])
aisles_in_depts = sorted_pad.groupby('aisle_name').head(1)[['dept_name', 'aisle_name']]
aisles_in_depts.groupby('dept_name').sum()

,aisle_name
dept_name,
alcohol,beers coolersred winesspecialty wines champagn...
babies,baby accessoriesbaby bath body carebaby food f...
bakery,bakery dessertsbreadbreakfast bakerybuns rolls...
beverages,cocoa drink mixescoffeeenergy sports drinksjui...
breakfast,breakfast bars pastriescerealgranolahot cereal...
bulk,bulk dried fruits vegetablesbulk grains rice d...
canned goods,canned fruit applesaucecanned jarred vegetable...
dairy eggs,buttercreameggsmilkother creams cheesespackage...
deli,fresh dips tapenadeslunch meatprepared mealspr...


In [201]:
# Explore products in aisles 'missing' and 'other'

missing_aisle = sorted_pad[sorted_pad['aisle_name'] == 'missing']

other_aisle = sorted_pad[sorted_pad['aisle_name'] == 'other']

In [202]:
list(other_aisle['product_name'])

['100% Pure Eucalyptus',
 '100% Pure Moisturizing Apricot Kernel Oil',
 '100% Pure Peppermint Oil',
 '100% Pure Tea Tree Oil',
 '38210 Full Jaw Cuticle Nipper',
 '3mg Melatonin Dietary Supplement Tablets - 240 CT',
 '5-HTP 100 Mg Vegetarian Capsules',
 '93/7 Ground Beef',
 'AA Rechargeable Nickel Metal Hydride Batteries',
 'Active Insect Repellant Aerosol',
 'Advanced Refreshing Gel',
 'Agar Agar Powder',
 'All Purpose Flour Blend',
 'All Purpose Precision Tip 2 Pack',
 'Almond Joy Mini Creamers',
 'Almost Naked Personal Lubricant',
 'Aloe Vera Plant',
 'Ant Killer Liquid',
 'Anti Bug Shake & Spray',
 'Anti-Fungal Liquid, Maximum Strength',
 'Anti-Stress Drink Assorted Flavor Packets',
 'Antibacterial Hand Soap',
 'Antibacterial Liquid Hand Soap Fresh Citrus',
 'Antifungal Clotrimazole Cream',
 'Aromatic Bitters',
 'Arouses & Intensifies Personal Lubricant',
 'Arrowroot Powder',
 'Arugula Rocket Salad',
 'Ashwagandha Anti-Stress And Energy Caplets',
 'Aspen Flower Essence Anxiety Homeo

In [203]:
list(missing_aisle['product_name'])

['100% Cauliflower & Sweet Potato Riced Veggies',
 '100% Grass-Fed No-Grain Strawberry Yogurt',
 '100% Juice Orange Pineapple',
 '100% Lactose Free Milk Vitamin D',
 '100% Natural Ginger Ale',
 '100% Natural Plain Baked Coconut Chips',
 '100% Orange Peach Mango Juice',
 '100% Paradise Blend Juice',
 '100% Probiotics Pineapple Mango Juice',
 '100% Pure Organic Coconut Water',
 '12 G. Protein Bar Coffee Chocolate',
 '12 Year Old Highland Single Malt Scotch Whisky',
 '2 Flavors in One Crazy Beans',
 '2 In 1 Fresh Berries Wild Raspberry Candle',
 '2% Milk Fat Creamy Small Curd Cottage Cheese',
 '2% Milk Fat Creamy Small Curd Peach Cottage Cheese',
 '2% Milkfat Lowfat Cottage Cheese Large Curd',
 '2% Organic Reduced Fat Milk',
 '2% Strawberry N Cream Greek Whips',
 '28 Qt Clear Storage Tote Clear With Blue Lid',
 '2nd Foods Carrot, Pear & Blackberry Baby Food',
 '3 Day Cleanse Total Body Reset Advanced Herbal Formula Capsules',
 '3 Tri Colored Organic Peppers',
 '3 in 1 Body Wash Shampoo & 

In [204]:
len(list(missing_aisle['product_name']))

1258

In [205]:
len(list(other_aisle['product_name']))

548

1806 is a lot of poorly-aisled products, more than I'd hope to see in an "other" column, and certainly "missing." It's hard to know at this point whether or not it's a good idea to try to give any of these better aisle classifications. That could be a waste of energy, or it could be that analyses end up relying heavily on aisles people purchase from and products being logically organized into aisles that match reasonable assumptions about products' relationships in which case plenty of these seem like they could actually fit well into existing or new aisles.

If I did that for the sake of analysis, though, it would subsequently require reclassification in the original products table and collaboration with the hypothetical products manager of instacart. There may be reasons for these classifications that I don't understand.

For now I'm keeping things the way they are, but changing things up in this area is something I'll keep in mind, despite my lack of domain knowledge, in case it seems especially helpful. 